## 0. 환경 설정

In [ ]:
!pip install neo4j-graphrag neo4j openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.3/92.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.6/296.6 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.0/383.0 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.0 MB/s eta 0:00:00


API 로그인 > API Key 생성 : https://openai.com/chatgpt/

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "sk-..."

## 1. GraphDB 불러오기

Neo4j Sandbox : https://sandbox.neo4j.com/

### 1-1. Neo4j 드라이버 설정

In [ ]:
from neo4j import GraphDatabase, basic_auth
import openai

driver = GraphDatabase.driver(
  "neo4j://54.224.232.177:7687",
  auth=basic_auth("neo4j", "breast-commanders-abrasion"))

```cypher
MATCH (m:Movie {title:$movie})<-[:RATED]-(u:User)-[:RATED]->(rec:Movie)
RETURN distinct rec.title AS recommendation LIMIT 20
```

- `(m:Movie {title:$movie})` : title이 $movie 인 노드
- `(rec:Movie)` : 이 노드는 rec이라는 변수로 지정
- `RETURN distinct rec.title` : rec 변수에 있는 노드의 title을 RETURN



In [ ]:
cypher_query = '''
MATCH (m:Movie {title:$movie})<-[:RATED]-(u:User)-[:RATED]->(rec:Movie)
RETURN distinct rec.title AS recommendation LIMIT 20
'''

with driver.session(database="neo4j") as session:
  results = session.read_transaction(
    lambda tx: tx.run(cypher_query,
                      movie="Crimson Tide").data())
  for record in results:
    print(record['recommendation'])

#driver.close()

<ipython-input-4-18901dc9fb7f>:7: DeprecationWarning: read_transaction has been renamed to execute_read
  results = session.read_transaction(


Mr. Holland's Opus
Apollo 13
Dead Man Walking
Seven (a.k.a. Se7en)
Heat
Get Shorty
Fugitive, The
Dave
Addams Family Values
True Lies
Speed
Lion King, The
Four Weddings and a Funeral
Forrest Gump
Star Trek: Generations
Shawshank Redemption, The
Stargate
Pulp Fiction
Outbreak
Miracle on 34th Street


## 2. GRAPH RAG 구현하기

### Text2Cypher Retriever로 만든 그래프 쿼리 결과 기반 RAG 방식

In [ ]:
from neo4j_graphrag.retrievers import Text2CypherRetriever
from neo4j_graphrag.llm import OpenAILLM

# 쿼리텍스트를 기반으로 Cypher 쿼리문을 생성하고, Retrieval 후 답변을 생성할 때 사용할 LLM
llm = OpenAILLM(model_name="gpt-4o", model_params={"temperature": 0})

### 1) Text2Cypher Retriever

Cypher 자동생성을 위해 필요한 정보 제공
- Neo4j DB Schema
- Input / Output(Query) 예시

#### Neo4j DB Schema

```
Node properties:
Person {name: STRING, born: INTEGER}
Movie {tagline: STRING, title: STRING, released: INTEGER}
Relationship properties:
ACTED_IN {roles: LIST}
REVIEWED {summary: STRING, rating: INTEGER}
The relationships:
(:Person)-[:ACTED_IN]->(:Movie)
(:Person)-[:DIRECTED]->(:Movie)
(:Person)-[:PRODUCED]->(:Movie)
(:Person)-[:WROTE]->(:Movie)
(:Person)-[:FOLLOWS]->(:Person)
(:Person)-[:REVIEWED]->(:Movie)
```

In [ ]:
from neo4j import GraphDatabase
from neo4j.time import Date

def get_node_datatype(value):
    """
        입력된 노드 Value의 데이터 타입을 반환하는 함수
    """
    if isinstance(value, str):
        return "STRING"
    elif isinstance(value, int):
        return "INTEGER"
    elif isinstance(value, float):
        return "FLOAT"
    elif isinstance(value, bool):
        return "BOOLEAN"
    elif isinstance(value, list):
        return f"LIST[{get_node_datatype(value[0])}]" if value else "LIST"
    elif isinstance(value, Date):
        return "DATE"
    else:
        return "UNKNOWN"

def get_schema(uri, user, password):
    """
        Graph DB의 정보를 받아 노드 및 관계의 프로퍼티를 추출하고 스키마 딕셔너리를 반환하는 함수
    """
    driver = GraphDatabase.driver(
        uri,
        auth=basic_auth(user, password))

    with driver.session() as session:
        # 노드 프로퍼티 및 타입 추출
        node_query = """
        MATCH (n)
        WITH DISTINCT labels(n) AS node_labels, keys(n) AS property_keys, n
        UNWIND node_labels AS label
        UNWIND property_keys AS key
        RETURN label, key, n[key] AS sample_value
        """
        nodes = session.run(node_query)

        # 관계 프로퍼티 및 타입 추출
        rel_query = """
        MATCH ()-[r]->()
        WITH DISTINCT type(r) AS rel_type, keys(r) AS property_keys, r
        UNWIND property_keys AS key
        RETURN rel_type, key, r[key] AS sample_value
        """
        relationships = session.run(rel_query)

        # 관계 유형 및 방향 추출
        rel_direction_query = """
        MATCH (a)-[r]->(b)
        RETURN DISTINCT labels(a) AS start_label, type(r) AS rel_type, labels(b) AS end_label
        ORDER BY start_label, rel_type, end_label
        """
        rel_directions = session.run(rel_direction_query)

        # 스키마 딕셔너리 생성
        schema = {"nodes": {}, "relationships": {}, "relations": []}

        for record in nodes:
            label = record["label"]
            key = record["key"]
            sample_value = record["sample_value"] # 데이터 타입을 추론하기 위한 샘플 데이터
            inferred_type = get_node_datatype(sample_value)
            if label not in schema["nodes"]:
                schema["nodes"][label] = {}
            schema["nodes"][label][key] = inferred_type

        for record in relationships:
            rel_type = record["rel_type"]
            key = record["key"]
            sample_value = record["sample_value"] # 데이터 타입을 추론하기 위한 샘플 데이터
            inferred_type = get_node_datatype(sample_value)
            if rel_type not in schema["relationships"]:
                schema["relationships"][rel_type] = {}
            schema["relationships"][rel_type][key] = inferred_type

        for record in rel_directions:
            start_label = record["start_label"][0]
            rel_type = record["rel_type"]
            end_label = record["end_label"][0]
            schema["relations"].append(f"(:{start_label})-[:{rel_type}]->(:{end_label})")

        return schema

def format_schema(schema):
    """
        스키마 딕셔너리를 LLM에 제공하기 위해 원하는 형태로 formatting 하는 함수
    """
    result = []

    # 노드 프로퍼티 출력
    result.append("Node properties:")
    for label, properties in schema["nodes"].items():
        props = ", ".join(f"{k}: {v}" for k, v in properties.items())
        result.append(f"{label} {{{props}}}")

    # 관계 프로퍼티 출력
    result.append("Relationship properties:")
    for rel_type, properties in schema["relationships"].items():
        props = ", ".join(f"{k}: {v}" for k, v in properties.items())
        result.append(f"{rel_type} {{{props}}}")

    # 관계 프로퍼티 출력
    result.append("The relationships:")
    for relation in schema["relations"]:
        result.append(relation)

    return "\n".join(result)

In [ ]:
# Neo4j DB Schema 제공
schema = get_schema("neo4j://54.224.232.177:7687","neo4j", "breast-commanders-abrasion")
neo4j_schema = format_schema(schema)
print(neo4j_schema)

Node properties:
Movie {url: STRING, runtime: INTEGER, revenue: INTEGER, budget: INTEGER, imdbRating: FLOAT, released: STRING, countries: LIST[STRING], languages: LIST[STRING], plot: STRING, imdbVotes: INTEGER, imdbId: STRING, year: INTEGER, poster: STRING, movieId: STRING, tmdbId: STRING, title: STRING}
Genre {name: STRING}
User {userId: STRING, name: STRING}
Actor {bornIn: STRING, born: DATE, died: DATE, tmdbId: STRING, imdbId: STRING, name: STRING, url: STRING, bio: STRING, poster: STRING}
Person {bornIn: STRING, born: DATE, died: DATE, tmdbId: STRING, imdbId: STRING, name: STRING, url: STRING, bio: STRING, poster: STRING}
Director {url: STRING, bornIn: STRING, bio: STRING, died: DATE, born: DATE, imdbId: STRING, name: STRING, poster: STRING, tmdbId: STRING}
Relationship properties:
RATED {rating: FLOAT, timestamp: INTEGER}
ACTED_IN {role: STRING}
DIRECTED {role: STRING}
The relationships:
(:Actor)-[:ACTED_IN]->(:Movie)
(:Actor)-[:DIRECTED]->(:Movie)
(:Actor)-[:ACTED_IN]->(:Movie)
(

#### Retriever 예시 작성

- 사용자 입력 : Which actors starred in the Toy Story? (Toy Story 영화에 어떤 배우들이 출연하였나요?)

- 자동 생성 Cypher 예시 : `MATCH (a:Actor)-[:ACTED_IN]->(m:Movie) WHERE m.title = 'Toy Story' RETURN a.name`

※ 추천시스템을 위한 예시 추가

※ 사용자의 한국어 질문에 맞춰 예시 수정



In [ ]:
# LLM INPUT / QUERY 예시 제공
examples = [
    "USER INPUT: 'Toy Story에 어떤 배우들이 출연하나요?' QUERY: MATCH (a:Actor)-[:ACTED_IN]->(m:Movie) WHERE m.title = 'Toy Story' RETURN a.name",
    "USER INPUT: 'Toy Story의 평균 평점은 몇점인가요?' QUERY: MATCH (u:User)-[r:RATED]->(m:Movie) WHERE m.title = 'Toy Story' RETURN AVG(r.rating)",

    """USER INPUT: '저는 Toy Story 영화를 좋아합니다. Toy Story를 재밌게 본 사람은 또 어떤 영화를 재밌게 봤나요?'
    QUERY: MATCH (m:Movie)<-[r:RATED]-(u:User)-[recr:RATED]->(userBasedRec:Movie)
    WHERE m.title = 'Toy Story' AND r.rating >= 4 AND recr.rating >= 4
    WITH userBasedRec, COUNT(recr) AS recCount, AVG(recr.rating) AS avgRating
    ORDER BY avgRating DESC, recCount DESC
    RETURN DISTINCT userBasedRec.title, avgRating, recCount
    LIMIT 10
    """,

    """USER INPUT: '저는 'Wizard of Oz, The' 와 같은 영화를 좋아합니다. 이 영화와 비슷한 영화 추천해줄 수 있나요?',
    QUERY: MATCH (m:Movie) WHERE m.title = 'Wizard of Oz, The'
    MATCH (m)-[:IN_GENRE]->(g:Genre)<-[:IN_GENRE]-(rec:Movie)
    WITH m, rec, count(*) AS gs

    OPTIONAL MATCH (m)<-[:ACTED_IN]-(a)-[:ACTED_IN]->(rec)
    WITH m, rec, gs, count(a) AS as

    OPTIONAL MATCH (m)<-[:DIRECTED]-(d)-[:DIRECTED]->(rec)
    WITH m, rec, gs, as, count(d) AS ds

    RETURN rec.title AS recommendation,
            rec.poster AS rec_poster,
            gs AS genre_similarity,
            as AS actor_similarity,
            ds AS director_similarity,
           (5*gs)+(3*as)+(4*ds) AS score
    ORDER BY score DESC LIMIT 10
    """,

    """USER INPUT: '영화 'Inception'과 비슷한 장르 혹은 비슷한 분위기의 영화를 추천해주세요.'
    QUERY: MATCH (m:Movie)-[:IN_GENRE]->(g:Genre)<-[:IN_GENRE]-(rec:Movie)
    WHERE m.title = 'Inception' WITH rec, collect(g.name) AS genres, count(*) AS commonGenres
    RETURN rec.title, genres, commonGenres ORDER BY commonGenres DESC LIMIT 10;"""
]

In [ ]:
# Text2CypherRetriever
retriever = Text2CypherRetriever(
    driver=driver,
    llm=llm,  # type: ignore
    neo4j_schema=neo4j_schema,
    examples=examples,
)

# LLM을 통해 Cypher 쿼리를 생성하여 Neo4j DB에 보내고, 그 결과를 반환 => 이 결과는 RAG에 활용됨
query_text = "Tom Hanks 가 어떤 영화에 출연했나요?"
search_result = retriever.search(query_text=query_text)

In [ ]:
search_result.items

[RetrieverResultItem(content="<Record m.title='Punchline'>", metadata=None),
 RetrieverResultItem(content="<Record m.title='Catch Me If You Can'>", metadata=None),
 RetrieverResultItem(content="<Record m.title='Dragnet'>", metadata=None),
 RetrieverResultItem(content="<Record m.title='Saving Mr. Banks'>", metadata=None),
 RetrieverResultItem(content="<Record m.title='Bachelor Party'>", metadata=None),
 RetrieverResultItem(content="<Record m.title='Volunteers'>", metadata=None),
 RetrieverResultItem(content="<Record m.title='Man with One Red Shoe, The'>", metadata=None),
 RetrieverResultItem(content="<Record m.title='Splash'>", metadata=None),
 RetrieverResultItem(content="<Record m.title='Big'>", metadata=None),
 RetrieverResultItem(content="<Record m.title='Nothing in Common'>", metadata=None),
 RetrieverResultItem(content="<Record m.title='Money Pit, The'>", metadata=None),
 RetrieverResultItem(content="<Record m.title='Toy Story of Terror'>", metadata=None),
 RetrieverResultItem(con

In [ ]:
query_text = "저는 Titanic을 좋아합니다. 비슷한 영화를 추천해줄 수 있나요?"
search_result = retriever.search(query_text=query_text)

In [ ]:
search_result.metadata['cypher']

"MATCH (m:Movie) WHERE m.title = 'Titanic'\nMATCH (m)-[:IN_GENRE]->(g:Genre)<-[:IN_GENRE]-(rec:Movie)\nWITH m, rec, count(*) AS gs\n\nOPTIONAL MATCH (m)<-[:ACTED_IN]-(a)-[:ACTED_IN]->(rec)\nWITH m, rec, gs, count(a) AS as\n\nOPTIONAL MATCH (m)<-[:DIRECTED]-(d)-[:DIRECTED]->(rec)\nWITH m, rec, gs, as, count(d) AS ds\n\nRETURN rec.title AS recommendation,\n       rec.poster AS rec_poster,\n       gs AS genre_similarity,\n       as AS actor_similarity,\n       ds AS director_similarity,\n       (5*gs)+(3*as)+(4*ds) AS score\nORDER BY score DESC LIMIT 10"

In [ ]:
print(search_result.metadata['cypher'])

MATCH (m:Movie) WHERE m.title = 'Titanic'
MATCH (m)-[:IN_GENRE]->(g:Genre)<-[:IN_GENRE]-(rec:Movie)
WITH m, rec, count(*) AS gs

OPTIONAL MATCH (m)<-[:ACTED_IN]-(a)-[:ACTED_IN]->(rec)
WITH m, rec, gs, count(a) AS as

OPTIONAL MATCH (m)<-[:DIRECTED]-(d)-[:DIRECTED]->(rec)
WITH m, rec, gs, as, count(d) AS ds

RETURN rec.title AS recommendation,
       rec.poster AS rec_poster,
       gs AS genre_similarity,
       as AS actor_similarity,
       ds AS director_similarity,
       (5*gs)+(3*as)+(4*ds) AS score
ORDER BY score DESC LIMIT 10


### 2) Retriever 기반 RAG 생성

In [ ]:
from neo4j_graphrag.generation import GraphRAG
# RAG 파이프라인 초기화
rag = GraphRAG(retriever=retriever, llm=llm)

답변에 사용한 Context 정보를 함께 확인 :
https://github.com/neo4j/neo4j-graphrag-python/blob/89411ca2c9ae7fdce63ee9678fe658b2e2ec30dd/src/neo4j_graphrag/generation/graphrag.py#L101

In [ ]:
# 질문하기
query_text = "Titanic과 비슷한 장르의 영화 추천해주세용."

response = rag.search(query_text=query_text, return_context = True)
print("==== [Text2Cypher 를 통해 자동생성한 Cypher] ====")
print(response.retriever_result.metadata['cypher'])
print("\n==== [생성된 Cypher를 기반으로 최종답변생성] ====")
print(response.answer)

==== [Text2Cypher 를 통해 자동생성한 Cypher] ====
MATCH (m:Movie)-[:IN_GENRE]->(g:Genre)<-[:IN_GENRE]-(rec:Movie)
WHERE m.title = 'Titanic'
WITH rec, collect(g.name) AS genres, count(*) AS commonGenres
RETURN rec.title, genres, commonGenres
ORDER BY commonGenres DESC
LIMIT 10;

==== [생성된 Cypher를 기반으로 최종답변생성] ====
Titanic과 비슷한 장르의 영화로는 다음과 같은 영화들을 추천드립니다:

1. Dirty Mary Crazy Larry
2. Shiri (Swiri)
3. Absolute Giganten
4. Eight Below
5. Kingdom of Heaven
6. Helen of Troy
7. Robin Hood
8. Legend of the Red Dragon (a.k.a. New Legend of Shaolin, The) (Hong Xi Guan: Zhi Shao Lin wu zu)
9. Casanova
10. House of Flying Daggers (Shi mian mai fu)

이 영화들은 모두 드라마, 로맨스, 액션 장르를 포함하고 있습니다.


In [ ]:
# 질문하기
query_text = "Toy Story와 The Godfather 영화를 좋아하는 사람은 또 어떤 영화를 좋아하나요?"

response = rag.search(query_text=query_text, return_context = True)
print("==== [Text2Cypher 를 통해 자동생성한 Cypher] ====")
print(response.retriever_result.metadata['cypher'])
print("\n==== [생성된 Cypher를 기반으로 최종답변생성] ====")
print(response.answer)

==== [Text2Cypher 를 통해 자동생성한 Cypher] ====
MATCH (m1:Movie)<-[r1:RATED]-(u:User)-[r2:RATED]->(m2:Movie)
WHERE m1.title IN ['Toy Story', 'The Godfather'] AND r1.rating >= 4 AND r2.rating >= 4
WITH m2, COUNT(r2) AS recCount, AVG(r2.rating) AS avgRating
ORDER BY avgRating DESC, recCount DESC
RETURN DISTINCT m2.title, avgRating, recCount
LIMIT 10

==== [생성된 Cypher를 기반으로 최종답변생성] ====
Toy Story와 The Godfather 영화를 좋아하는 사람들은 다음과 같은 영화를 좋아할 수 있습니다:

1. Killing Fields, The
2. Rosencrantz and Guildenstern Are Dead
3. Sherlock Holmes: A Game of Shadows
4. Paris Is Burning
5. Stardust
6. Anne Frank Remembered
7. Down by Law
8. I Shot Andy Warhol
9. You Can't Take It with You
10. Blink

이 영화들은 모두 평균 평점이 5.0으로 높은 평가를 받은 영화들입니다.


## 3. Gradio 로 챗봇 배포하기

In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 7.6 MB/s eta 0:00:00


In [ ]:
import gradio as gr
from gradio.themes.base import Base

class Seafoam(Base):
    pass
seafoam = Seafoam()

with gr.Blocks(theme=seafoam) as demo: #'JohnSmith9982/small_and_pretty'
    def default_llm(message):
        prompt_text = f"""
        당신은 영화 추천 시스템을 탑재한 챗봇입니다. user_input 에 대답하되, 사용자에게 좋아하는 영화나 장르 등을 말해보라고 권고해보세요.
        user_input : {message}
        """
        return llm.invoke(prompt_text).content

    def intent_detection(message):
        prompt_text = f"""
        주어진 query_text 가 영화 추천을 받기 위한 본격적인 질문으로 보이면 True를 반환하고, 그렇지 않으면 False를 반환해주세요.
        예시 : [("query_text": "안녕 반가워", "answer": "False"), ("query_text": "네가 영화추천을 그렇게 잘한다며?", "answer": "False"), ("query_text": "Titanic 영화와 비슷한 장르의 영화를 추천해줄래?", "answer": "True")]
        query_text : {message}
        """
        return llm.invoke(prompt_text).content == 'True'

    def response(message, chat_history):
        #### INTENT DETECTION ####
        if(intent_detection(message)):
            #### ANSWER ####
            rag_result = rag.search(query_text=message
                                    + "(Please also provide evidence for how you used context in your answer. YOU MUST ANSWER in KOREAN PLEASE.)"
                                    , return_context = True)
            chat_history.append((message, rag_result.answer))
            return chat_history, rag_result.retriever_result.metadata['cypher'], rag_result.retriever_result.items
        else:
            llm_result = default_llm(message)
            chat_history.append((message, llm_result))
            return chat_history, "영화 관련 질문이 아니었어요.", "영화 관련 질문이 아니었어요."

    with gr.Row():
        with gr.Column(scale=4):
            gr.HTML("""<div style="text-align: center; max-width: 1000px; margin: 10px auto;">
                <div>
                    <h1>Graph RAG 챗봇 !</h1>
                </div>
                <p style="margin-bottom: 10px; font-size: 95%">
                    💭 Graph DB의 영화 리뷰 데이터셋을 기반으로 답변합니다. 답변에 사용한 DB 조회 결과를 함께 확인해보세요. </a>
                </p>
            </div>""")


    with gr.Row():
        with gr.Column(scale = 1):
            generated_query = gr.Textbox(label="생성된 Cypher 쿼리")
            query_result = gr.Textbox(label="쿼리 조회 결과")
        with gr.Column(scale = 3):
            chatbot = gr.Chatbot()
            msg = gr.Textbox(placeholder="어떤 영화를 추천받고 싶으신가요? (원하는 장르나 재밌게 봤던 영화를 함께 말하면 도움이 됩니다.)", label="입력")
            examples = gr.Examples(
                examples=[
                    "저는 'Net, The' 와 같은 영화를 좋아합니다. 이 영화와 비슷한 영화 추천해줄 수 있나요?",
                    "영화 'Inception'과 비슷한 장르 혹은 비슷한 분위기의 영화를 추천해주세요."
                ],
                inputs=[msg],
            )
            with gr.Row():
                gr.HTML("""<div style="text-align: center; max-width: 500px; margin: 0px auto;">
                    <div>
                        <h1>  </h1>
                    </p>
                </div>""")
                gr.HTML("""<div style="text-align: center; max-width: 500px; margin: 0px auto;">
                    <div>
                        <h1>  </h1>
                    </p>
                </div>""")
                btn = gr.Button("Submit", variant="primary")
                clear = gr.Button("Clear")

    btn.click(fn=response, inputs=[msg, chatbot], outputs=[chatbot, generated_query, query_result])
    msg.submit(response, [msg, chatbot], [chatbot, generated_query, query_result])

    clear.click(lambda: None, None, msg, queue=False)

demo.launch(debug=True, share=True)#debug=True, share=True

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://7c169af5eec5323445.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://7c169af5eec5323445.gradio.live
